## Part 1: Preprocessing

In [16]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [17]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [18]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [81]:
# Create a list of at least 10 column names to use as X data
selected_columns = [
    'Education',
    'EducationField',
    'Age',
    'DistanceFromHome',
    'JobSatisfaction',
    'OverTime',
    'StockOptionLevel',
    'WorkLifeBalance',
    'YearsAtCompany',
    'YearsSinceLastPromotion',
    'NumCompaniesWorked',
    'JobInvolvement',
    'PerformanceRating'
    ]

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.dtypes


,0
Education,int64
EducationField,object
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64


In [82]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)


In [83]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

# Find categorical columns that need to be encoded
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()
for col in categorical_cols:
    print(f"{X_train[col].value_counts()}")
    print(f"{col} null values: {X_train[col].isna().sum()}")
    print('-'*25)


EducationField
Life Sciences       451
Medical             344
Marketing           118
Technical Degree    102
Other                67
Human Resources      20
Name: count, dtype: int64
EducationField null values: 0
-------------------------
OverTime
No     798
Yes    304
Name: count, dtype: int64
OverTime null values: 0
-------------------------


In [84]:
# Create a OneHotEncoder for the categorical columns taking into account the OverTime binary Yes, No values
ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='if_binary')
ohe_encoder.set_output(transform="pandas")

# Fit the encoder to the training data
ohe_encoder.fit(X_train[categorical_cols])

# Encode the training and testing data
X_train_encoded = ohe_encoder.transform(X_train[categorical_cols])
X_test_encoded = ohe_encoder.transform(X_test[categorical_cols])


In [85]:
# Concatenate encoded df's with original data and drop orig columns
X_train_encoded = pd.concat([X_train.drop(columns=categorical_cols), X_train_encoded], axis=1)
X_test_encoded = pd.concat([X_test.drop(columns=categorical_cols), X_test_encoded], axis=1)

In [30]:
# Sanity check
X_train_encoded.head()

,Education,Age,DistanceFromHome,JobSatisfaction,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked,JobInvolvement,PerformanceRating,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,OverTime_Yes
464,3,37,1,4,0,4,6,1,4,3,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0
512,4,27,3,4,0,3,4,1,0,2,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0
480,4,30,12,1,1,4,1,0,1,2,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0
680,4,36,7,4,0,3,6,0,2,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1023,2,56,1,1,1,4,3,1,2,3,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [86]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train_encoded)

# Scale the training and testing data
X_train_processed = scaler.transform(X_train_encoded)
X_test_processed = scaler.transform(X_test_encoded)


In [88]:
# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
dept_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_dept_encoded = dept_encoder.transform(y_train[['Department']])
y_test_dept_encoded = dept_encoder.transform(y_test[['Department']])

y_train_dept_encoded

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [89]:
# Create a OneHotEncoder for the Attrition column use if_binary to encode to a single column
attr_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='if_binary')

# Fit the encoder to the training data
attr_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attr_encoded = attr_encoder.transform(y_train[['Attrition']])
y_test_attr_encoded = attr_encoder.transform(y_test[['Attrition']])

y_train_attr_encoded


array([[0.],
       [0.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]])

## Create, Compile, and Train the Model

In [90]:
# Find the number of columns in the X training data
num_input_cols = X_train_processed.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_input_cols,), name='input_layer')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name='shared_layer1')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu', name='shared_layer2')(shared_layer1)


In [93]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu', name='department_hidden')(shared_layer2)

# Create the output layer (use softmax b/c the 3 department values are mutually exclusive)
department_output = layers.Dense(y_train_dept_encoded.shape[1], activation='softmax', name='department_output')(department_hidden)


In [94]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu', name='attrition_hidden')(shared_layer2)

# Create the output layer (use sigmoid b/c attrition is binary)
attrition_output = layers.Dense(y_train_attr_encoded.shape[1], activation='sigmoid', name='attrition_output')(attrition_hidden)


In [95]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model (use categorical_crossentropy for dept and binary_crossentropy for attr)
model.compile(
    optimizer='adam',
    loss={'department_output': 'categorical_crossentropy',
          'attrition_output': 'binary_crossentropy'},
    metrics={'department_output': 'accuracy',
             'attrition_output': 'accuracy'}
)

# Summarize the model
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 18)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer1 (Dense)     │ (None, 64)             │          1,216 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer2 (Dense)     │ (None, 128)            │          8,320 │ shared_layer1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_hidden (Dense) │ (None, 32)             │          4,128 │ shared_layer2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_hidden (Dense)  │ (None, 32)             │          4,128 │ shared_layer2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ department_hidden[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 1)              │             33 │ attrition_hidden[0][0] │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,924 (70.02 KB)

 Trainable params: 17,924 (70.02 KB)

 Non-trainable params: 0 (0.00 B)

In [96]:
# Train the model
model.fit(
    X_train_processed,
    {
        'department_output': y_train_dept_encoded,
        'attrition_output': y_train_attr_encoded
    },
    epochs=100
)


Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - attrition_output_accuracy: 0.7915 - department_output_accuracy: 0.6769 - loss: 1.4103
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8392 - department_output_accuracy: 0.7453 - loss: 1.0884
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8435 - department_output_accuracy: 0.7846 - loss: 0.9871
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8656 - department_output_accuracy: 0.7750 - loss: 0.9144
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8714 - department_output_accuracy: 0.7747 - loss: 0.8922
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8754 - department_output_accuracy: 0.7952 - loss: 0.8379
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8830 - department_output_accuracy: 0.7749 - loss: 0.8559
Epoch 8/100
35/35 ━━━━━━━━━

In [97]:
# Evaluate the model with the testing data
test_results = model.evaluate(
    X_test_processed,
    {
        'department_output': y_test_dept_encoded,
        'attrition_output': y_test_attr_encoded
    }
)
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.7849 - department_output_accuracy: 0.5967 - loss: 3.9650  


[3.827521324157715, 0.8097826242446899, 0.6114130616188049]

In [98]:
# Print the accuracy for both department and attrition
print(f"Department predictions accuracy: {test_results[2]}")
print(f"Attrition predictions accuracy: {test_results[1]}")


Department predictions accuracy: 0.6114130616188049
Attrition predictions accuracy: 0.8097826242446899


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.